In [276]:
import duckdb as ddb
import pandas as pd

con = ddb.connect(database=":memory:")

con.sql("""
CREATE TABLE program_beslut AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-beslut.csv';
CREATE TABLE program_kommun AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-diarie_kommun.csv';

CREATE TABLE kurser_beslut AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-beslut.csv';
CREATE TABLE kurser_kommun AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-diarie_kommun.csv';

CREATE TABLE kommun_lan AS
    FROM 'data/kommunlankod-2025.csv';       
""")

In [277]:
con.sql("""select * from kurser_beslut""").describe()
# con.sql("""select * from kurser_beslut""").columns

┌─────────┬───────────────┬──────────────────┬─────────┬─────────────────────────────────┬─────────────────────────────────────────────────┬───────────────────┬───────────────────┬────────────────────┬──────────────────────────┬────────────────────┬──────────────┬───────────┬────────────────────┬────────────────┐
│  aggr   │ Diarienummer  │ Ansökningsomgång │ Beslut  │      Utbildningsanordnare       │                 Utbildningsnamn                 │ Utbildningsområde │     YH-poäng      │ Beviljade platser  │ Beviljade platser totalt │       Start        │     Län      │  Kommun   │   Antal kommuner   │ Flera kommuner │
│ varchar │    varchar    │      double      │ varchar │             varchar             │                     varchar                     │      varchar      │      double       │       double       │          double          │       double       │   varchar    │  varchar  │       double       │    varchar     │
├─────────┼───────────────┼──────────────────┼─────────

In [278]:
def get_utbildningsanordnare(con):
    rel = con.sql("""
    select distinct Utbildningsanordnare
    from kurser_beslut
    order by Utbildningsanordnare
    """)
    return [i[0] for i in rel.fetchall()]


get_utbildningsanordnare(con)

['ABF Göteborg Vuxenutbildning AB',
 'AGSTU AB',
 'ALEA Kompetenshöjning AB',
 'AVT AB',
 'Ai Högskolan Sverige AB',
 'Almost There AB',
 'Amsac AB',
 'Angered folkhögskola',
 'Arboga kommun, Vuxenutbildningen',
 'Arvika Kommun',
 'Bergsskolan Kompetensutveckling AB',
 'Blue Peak AB',
 'Borås Stad - Yrkeshögskolan i Borås',
 'Båstads kommun Akademi Båstad',
 'Cady training utbildningar AB',
 'Campus Ljungby AB',
 'Campus Nyköping',
 'Campus Nynäshamn',
 'Campus Varberg/Varbergs kommun',
 'Changemaker Educations AB',
 'Chas Academy AB',
 'Dans i Göteborg AB',
 'Digitala lyftet i Sverige AB',
 'Dille Gård AB',
 'EBT Academy AB',
 'EC Utbildning AB Göteborg',
 'EC Utbildning AB Malmö',
 'EC Utbildning AB Stockholm',
 'EC Utbildning AB Örebro',
 'East Sweden Business School AB',
 'EdUP Group AB',
 'Edugrade AB',
 'Elevera Education Group',
 'Enköpings kommun, Yrkeshögskolan i Enköping',
 'Equine Academy Sweden AB',
 'Eriksmåla Ridklubb',
 'Falkenbergs kommun',
 'Fastighetsakademin Sverige 

In [279]:
def create_df_utbildningsomrade_beviljade(con, utbildningsanordnare: list[str] | None = None, year=2024):
    rel = con.sql(
        """
    select
        Utbildningsområde,
        count(*) as Beviljade
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = true
    group by Utbildningsområde
    order by Beviljade desc, Utbildningsområde
    ;""",
        params={"year": year, "utb": utbildningsanordnare},
    )
    return rel.df().set_index(["Utbildningsområde"])


create_df_utbildningsomrade_beviljade(con)

,Beviljade
Utbildningsområde,
"Ekonomi, administration och försäljning",171
Data/IT,111
Teknik och tillverkning,106
Samhällsbyggnad och byggteknik,93
Hälso- och sjukvård samt socialt arbete,81
"Hotell, restaurang och turism",19
"Lantbruk, djurvård, trädgård, skog och fiske",18
Pedagogik och undervisning,15
Journalistik och information,11


In [280]:
create_df_utbildningsomrade_beviljade(con, ["Stiftelsen Stockholms Tekniska Institut"])

,Beviljade
Utbildningsområde,
Data/IT,7
Samhällsbyggnad och byggteknik,6


In [281]:
def create_df_utbildningsomrade_avslag(con, utbildningsanordnare: list[str] | None = None, year=2024):
    rel = con.sql(
        """
    select
        Utbildningsområde,
        count(*) as Avslag
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = false
    group by Utbildningsområde
    order by Avslag desc, Utbildningsområde
    ;""",
        params={"year": year, "utb": utbildningsanordnare},
    )
    return rel.df().set_index("Utbildningsområde")


create_df_utbildningsomrade_avslag(con)

,Avslag
Utbildningsområde,
"Ekonomi, administration och försäljning",112
Data/IT,84
Hälso- och sjukvård samt socialt arbete,53
Teknik och tillverkning,51
Samhällsbyggnad och byggteknik,41
"Lantbruk, djurvård, trädgård, skog och fiske",24
Pedagogik och undervisning,18
Säkerhetstjänster,11
"Hotell, restaurang och turism",9


In [282]:
create_df_utbildningsomrade_avslag(con, ["Stiftelsen Stockholms Tekniska Institut"])

,Avslag
Utbildningsområde,
Data/IT,1
Samhällsbyggnad och byggteknik,1


In [283]:
def create_df_utbildningsomrade_beslut(con, beslut=True, utbildningsanordnare: list[str] | None = None, year=2024):
    rel = con.sql(
        """
    select
        Utbildningsområde,
        count(*) as Beslut
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = $beslut
    group by Utbildningsområde
    order by Beslut desc, Utbildningsområde
    ;""",
        params={"beslut": beslut, "year": year, "utb": utbildningsanordnare},
    )

    df_rel: pd.Dataframe = rel.df().set_index("Utbildningsområde")
    return df_rel.rename(columns={"Beslut": "Beviljade" if beslut else "Avslag"})


create_df_utbildningsomrade_beslut(con, beslut=False)

,Avslag
Utbildningsområde,
"Ekonomi, administration och försäljning",112
Data/IT,84
Hälso- och sjukvård samt socialt arbete,53
Teknik och tillverkning,51
Samhällsbyggnad och byggteknik,41
"Lantbruk, djurvård, trädgård, skog och fiske",24
Pedagogik och undervisning,18
Säkerhetstjänster,11
"Hotell, restaurang och turism",9


In [284]:
def get_kurser_antal(con):
    rel = con.sql("""
    select
        (select count(*) from kurser_beslut) as "Kurs-ansökningar",
        (select count(*) from kurser_kommun) as "Kurs-ansökningar, alla kommuner"
    """)
    return rel.fetchone()


get_kurser_antal(con)

(1090, 1202)

In [285]:
def get_utbildningsomraden(con):
    rel = con.sql("""
    select distinct Utbildningsområde
    from kurser_beslut
    order by Utbildningsområde
    """)
    return [i[0] for i in rel.fetchall()]


get_utbildningsomraden(con)

['Data/IT',
 'Ekonomi, administration och försäljning',
 'Friskvård och kroppsvård',
 'Hotell, restaurang och turism',
 'Hälso- och sjukvård samt socialt arbete',
 'Journalistik och information',
 'Juridik',
 'Kultur, media och design',
 'Lantbruk, djurvård, trädgård, skog och fiske',
 'Pedagogik och undervisning',
 'Samhällsbyggnad och byggteknik',
 'Säkerhetstjänster',
 'Teknik och tillverkning',
 'Transporttjänster']

In [286]:
def create_df_kurser_beviljade(con, utbildningsomrade: list[str] | None = None):
    rel = con.sql(
        """
    select
        Utbildningsanordnare,
        -- Utbildningsområde,
        count(*) as Ansökningar,
        sum(case when Beslut = true then 1 else 0 end)::integer as Beviljade,
        round(100.00 * sum(case when Beslut = true then 1 else 0 end) / count(*),1) as "Beviljade %",
    from kurser_beslut
    where ( $areas IS NULL OR Utbildningsområde = ANY($areas) )
    group by Utbildningsanordnare, Utbildningsområde
    order by "Beviljade %" desc, Ansökningar desc, Utbildningsanordnare
    """,
        params={"areas": utbildningsomrade},
    )
    return rel.df().set_index("Utbildningsanordnare")


create_df_kurser_beviljade(con, ["Data/IT"]).head(20)

,Ansökningar,Beviljade,Beviljade %
Utbildningsanordnare,,,
IT-Högskolan Sverige AB,11,11,100.0
Nackademin AB,9,9,100.0
YH Akademin AB,8,8,100.0
Handelsakademin i Göteborg AB,6,6,100.0
EC Utbildning AB Göteborg,4,4,100.0
AGSTU AB,2,2,100.0
Folkuniversitetet - Stiftelsen Kursverksamheten vid Uppsala universitetet,2,2,100.0
Hermods AB Örebro,2,2,100.0
Hyper Island Program AB - Karlskrona,2,2,100.0


In [287]:
create_df_kurser_beviljade(con, None).head(20)

,Ansökningar,Beviljade,Beviljade %
Utbildningsanordnare,,,
KompetensUtvecklingsInstitutet Sverige AB,15,15,100.0
Nackademin AB,13,13,100.0
IT-Högskolan Sverige AB,11,11,100.0
Nackademin AB,9,9,100.0
Sverigehälsan AB,9,9,100.0
Lexicon Yrkeshögskola AB,8,8,100.0
Medieinstitutet i Sverige AB,8,8,100.0
YH Akademin AB,8,8,100.0
One Academy AB,7,7,100.0


In [288]:
con.sql("""select * from program_beslut""").describe()

┌─────────┬───────────────────┬────────────────────────────────────────────────────────────────────┬──────────────┬──────────────┬────────────────────┬────────────────┬───────────────────┬────────────┬────────────────────┬─────────────────────────────────┬───────────────┬───────────────────────────┬───────────────────────────────┬──────────────────────┬──────────────────────────┬─────────────────────────────────────┬────────────────────┬───────────────┬─────────┐
│  aggr   │ Utbildningsområde │                          Utbildningsnamn                           │     Län      │    Kommun    │   Antal kommuner   │ Flera kommuner │     YH-poäng      │ Studieform │    Studietakt %    │      Utbildningsanordnare       │ Huvudmannatyp │ Sökta utbildningsomgångar │ Beviljade utbildningsomgångar │ Sökta platser totalt │ Beviljade platser totalt │ Sökta platser per utbildningsomgång │  Ansökningsomgång  │ Diarienummer  │ Beslut  │
│ varchar │      varchar      │                              var

In [289]:
def create_df_lan_utbildningar(con):
    rel = con.sql("""
    select
        Län,
        count(distinct Diarienummer) as "Antal beviljade utbildningar",
    from program_beslut
    where
        Ansökningsomgång = 2024
        and beslut = true
    group by Län
    order by "Antal beviljade utbildningar" desc
    """)
    return rel.df().set_index("Län")


create_df_lan_utbildningar(con).head(20)

,Antal beviljade utbildningar
Län,
Stockholm,82
Flera kommuner,65
Västra Götaland,63
Skåne,37
Östergötland,16
Jönköping,13
Örebro,10
Västernorrland,6
Gävleborg,5


In [290]:
def create_df_lan_utbildningsanordnare(
    con,
    lan: list[str] | None = None,
    beslut: bool | None = None,
    year=2024,
):
    if not lan:
        lan = None
    rel = con.sql(
        """
    select
        Län,
        count(distinct Utbildningsanordnare) as "Antal utbildningsanordnare",
    from program_beslut
    where
        Ansökningsomgång = $year
        and ( $beslut IS NULL OR Beslut = $beslut )
        and ( $lan IS NULL OR Län = ANY($lan) )
    group by Län
    order by "Antal utbildningsanordnare" desc
    """,
        params={"year": year, "lan": lan, "beslut": beslut},
    )
    return rel.df().set_index("Län")


create_df_lan_utbildningsanordnare(con, []).head(20)

,Antal utbildningsanordnare
Län,
Stockholm,77
Flera kommuner,75
Västra Götaland,57
Skåne,52
Västerbotten,17
Östergötland,15
Västmanland,13
Örebro,12
Västernorrland,11


In [291]:
def create_df_lan_utbildningar(
    con,
    lan: list[str] | None = None,
    beslut: bool | None = None,
    year=2024,
):
    if not lan:
        lan = None

    rel = con.sql(
        """
    select
        Län,
        count(distinct Diarienummer) as "Utbildningar, antal ansökta",
    from program_beslut
    where
        Ansökningsomgång = $year
        and ( $beslut IS NULL OR Beslut = $beslut )
        and "Flera kommuner" = False
        and ( $lan IS NULL OR Län = ANY($lan) )
    group by Län
    order by "Utbildningar, antal ansökta" desc
    """,
        params={"year": year, "lan": lan, "beslut": beslut},
    )

    df_rel: pd.Dataframe = rel.df().set_index("Län")
    if beslut is not None:
        df_rel = df_rel.rename(
            columns={
                "Utbildningar, antal ansökta": "Utbildningar, antal beviljade" if beslut else "Utbildningar, antal avslag",
            }
        )
    return df_rel


create_df_lan_utbildningar(con, []).head(20)

,"Utbildningar, antal ansökta"
Län,
Stockholm,313
Västra Götaland,212
Skåne,137
Östergötland,41
Jönköping,39
Västernorrland,28
Västerbotten,26
Västmanland,25
Örebro,24


In [292]:
create_df_lan_utbildningar(con, beslut=False).head(20)

,"Utbildningar, antal avslag"
Län,
Stockholm,231
Västra Götaland,149
Skåne,100
Jönköping,26
Östergötland,25
Västerbotten,22
Västernorrland,22
Västmanland,20
Örebro,14


In [293]:
create_df_lan_utbildningar(con, ["Skåne", "Norrbotten"]).head(20)

,"Utbildningar, antal ansökta"
Län,
Skåne,137
Norrbotten,15


In [294]:
con.sql("""select * from program_kommun""").describe()

┌─────────┬───────────────┬──────────────┬──────────────┬─────────┐
│  aggr   │ Diarienummer  │    Kommun    │     Län      │ Beslut  │
│ varchar │    varchar    │   varchar    │   varchar    │ varchar │
├─────────┼───────────────┼──────────────┼──────────────┼─────────┤
│ count   │ 8589          │ 8589         │ 8589         │ 8589    │
│ mean    │ NULL          │ NULL         │ NULL         │ NULL    │
│ stddev  │ NULL          │ NULL         │ NULL         │ NULL    │
│ min     │ MYH 2020/1698 │ Alingsås     │ Blekinge     │ false   │
│ max     │ MYH 2024/4278 │ Östra Göinge │ Östergötland │ true    │
│ median  │ NULL          │ NULL         │ NULL         │ NULL    │
└─────────┴───────────────┴──────────────┴──────────────┴─────────┘

In [295]:
con.sql("""select * from kommun_lan""").describe()

┌─────────┬────────────┬──────────────┬───────────────────┬────────────────────┐
│  aggr   │   Kommun   │     Län      │     Län, namn     │        Kod         │
│ varchar │  varchar   │   varchar    │      varchar      │       double       │
├─────────┼────────────┼──────────────┼───────────────────┼────────────────────┤
│ count   │ 290        │ 290          │ 290               │              290.0 │
│ mean    │ NULL       │ NULL         │ NULL              │ 1382.0896551724138 │
│ stddev  │ NULL       │ NULL         │ NULL              │  715.5540783148119 │
│ min     │ Ale        │ Blekinge     │ Blekinge län      │              114.0 │
│ max     │ Övertorneå │ Östergötland │ Östergötlands län │             2584.0 │
│ median  │ NULL       │ NULL         │ NULL              │             1439.5 │
└─────────┴────────────┴──────────────┴───────────────────┴────────────────────┘

In [296]:
con.sql(
    """
    select p.*, k.Län
    from program_kommun as p
    left join kommun_lan as k
        on p.Kommun = k.Kommun
    where p.Län != k.Län
    ;""").df()

,Diarienummer,Kommun,Län,Beslut,Län_1


In [297]:
con.sql(
    """
    with kurser_alla_kommuner as (

    select
        "Diarienummer",
        "Ansökningsomgång",
        "Beslut",
        "Utbildningsanordnare",
        "Utbildningsnamn",
        "Utbildningsområde",
        "YH-poäng",
        "Beviljade platser",
        "Beviljade platser totalt",
        "Start",
        "Län",
        "Kommun",
        "Antal kommuner",
        "Flera kommuner"
    from kurser_beslut kb
    where
        "Flera kommuner" is not TRUE
    
    union all

    select
        kb."Diarienummer",
        kb."Ansökningsomgång",
        kb."Beslut",
        kb."Utbildningsanordnare",
        kb."Utbildningsnamn",
        kb."Utbildningsområde",
        kb."YH-poäng",
        kb."Beviljade platser",
        kb."Beviljade platser totalt",
        kb."Start",
        kk."Län",                -- kurser_kommun
        kk."Kommun",             -- kurser_kommun
        kb."Antal kommuner",
        kb."Flera kommuner"
    from kurser_beslut kb
    join kurser_kommun kk
        on kk."Diarienummer" = kb."Diarienummer"
     where
        kb."Flera kommuner" is TRUE

    )

    select *
    from kurser_alla_kommuner
    order by "Diarienummer"
    ;""").df()

,Diarienummer,Ansökningsomgång,Beslut,Utbildningsanordnare,Utbildningsnamn,Utbildningsområde,YH-poäng,Beviljade platser,Beviljade platser totalt,Start,Län,Kommun,Antal kommuner,Flera kommuner
0,MYH 2024/1000,2024,False,Changemaker Educations AB,Futuregames Accessible Game Production,Juridik,25,0,0,2024,Stockholm,Stockholm,3,True
1,MYH 2024/1000,2024,False,Changemaker Educations AB,Futuregames Accessible Game Production,Juridik,25,0,0,2024,Norrbotten,Boden,3,True
2,MYH 2024/1000,2024,False,Changemaker Educations AB,Futuregames Accessible Game Production,Juridik,25,0,0,2024,Skåne,Malmö,3,True
3,MYH 2024/1001,2024,False,Changemaker Educations AB,Futuregames Inclusive Game Development,"Ekonomi, administration och försäljning",25,0,0,2024,Skåne,Malmö,3,True
4,MYH 2024/1001,2024,False,Changemaker Educations AB,Futuregames Inclusive Game Development,"Ekonomi, administration och försäljning",25,0,0,2024,Norrbotten,Boden,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,MYH 2024/997,2024,False,INSU AB,"Säkerhetsprojektering- brand-, inbrott-, kamer...",Säkerhetstjänster,95,0,0,2024,Södermanland,Nyköping,1,False
1458,MYH 2024/998,2024,True,Campus Nyköping,Hälsofrämjande och pedagogiska insatser med häst,"Lantbruk, djurvård, trädgård, skog och fiske",50,24,24,2024,Södermanland,Nyköping,1,False
1459,MYH 2024/999,2024,False,Changemaker Educations AB,Futuregames Accessible Game Design,Data/IT,25,0,0,2024,Skåne,Malmö,3,True
1460,MYH 2024/999,2024,False,Changemaker Educations AB,Futuregames Accessible Game Design,Data/IT,25,0,0,2024,Norrbotten,Boden,3,True


In [298]:
def create_rel_kurser_alla_kommuner(con, years: list[int] | int | None = None, *, distinct=False):
    if not years:
        years = None
    elif isinstance(years, int):
        years = [years]

    rel_all = con.sql(
        """
        with kurser_alla_kommuner as (

        select
            "Diarienummer",
            "Ansökningsomgång",
            "Beslut",
            "Utbildningsanordnare",
            "Utbildningsnamn",
            "Utbildningsområde",
            "YH-poäng",
            "Beviljade platser",
            "Beviljade platser totalt",
            "Start",
            "Län",
            "Kommun",
            "Antal kommuner",
            "Flera kommuner"
        from kurser_beslut kb
        where
            "Flera kommuner" is not TRUE
            -- and kb."Ansökningsomgång" = 2024
            and ( $year IS NULL OR kb."Ansökningsomgång" = ANY($year) )
        
        union all

        select
            kb."Diarienummer",
            kb."Ansökningsomgång",
            kb."Beslut",
            kb."Utbildningsanordnare",
            kb."Utbildningsnamn",
            kb."Utbildningsområde",
            kb."YH-poäng",
            kb."Beviljade platser",
            kb."Beviljade platser totalt",
            kb."Start",
            kk."Län",                -- kurser_kommun
            kk."Kommun",             -- kurser_kommun
            kb."Antal kommuner",
            kb."Flera kommuner"
        from kurser_beslut kb
        -- join (select distinct "Diarienummer", "Kommun", "Län" from kurser_kommun) kk
        join kurser_kommun kk
            on kk."Diarienummer" = kb."Diarienummer"
        where
            kb."Flera kommuner" is TRUE
            and ( $year IS NULL OR kb."Ansökningsomgång" = ANY($year) )

        )
        select
            -- distinct on ("Diarienummer", "Kommun")
            *
        from kurser_alla_kommuner
        order by "Diarienummer"
        ;""",
        params={"year": years})

    if distinct:
        rel_distinct = con.sql(
            """
            select distinct on ("Diarienummer", "Kommun")
                "Diarienummer",
                "Ansökningsomgång",
                "Beslut",
                "Utbildningsanordnare",
                "Utbildningsnamn",
                "Utbildningsområde",
                "YH-poäng",
                -- "Beviljade platser",
                "Beviljade platser totalt",
                -- "Start",
                "Län",
                "Kommun",
                "Antal kommuner",
                "Flera kommuner"
            from rel_all
            ;""")

        return rel_distinct

    return rel_all

create_rel_kurser_alla_kommuner(con, years=2024, distinct=True).df()

,Diarienummer,Ansökningsomgång,Beslut,Utbildningsanordnare,Utbildningsnamn,Utbildningsområde,YH-poäng,Beviljade platser totalt,Län,Kommun,Antal kommuner,Flera kommuner
0,MYH 2024/1007,2024,False,Plushögskolan AB - Affärshögskolan,Hållbara inköps- och upphandlingsprocesser,"Ekonomi, administration och försäljning",45,0,Västra Götaland,Göteborg,5,True
1,MYH 2024/1009,2024,True,Plushögskolan AB - Affärshögskolan,Strategisk hållbarhetsutveckling,"Ekonomi, administration och försäljning",45,100,Örebro,Örebro,4,True
2,MYH 2024/1016,2024,True,Plushögskolan AB - Vårdyrkeshögskolan,"E-hälsa, digitalisering och välfärdsteknik ino...",Hälso- och sjukvård samt socialt arbete,50,50,Västra Götaland,Åmål,5,True
3,MYH 2024/1019,2024,True,Plushögskolan AB - Vårdyrkeshögskolan,Psykisk hälsa och suicidprevention,Hälso- och sjukvård samt socialt arbete,30,50,Västra Götaland,Uddevalla,5,True
4,MYH 2024/1021,2024,False,Plushögskolan AB - Vårdyrkeshögskolan,SRHR och funktionsnedsättning,Hälso- och sjukvård samt socialt arbete,50,0,Västra Götaland,Göteborg,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1197,MYH 2024/968,2024,False,INSU AB,Sprinklerprojektering,Samhällsbyggnad och byggteknik,80,0,Södermanland,Katrineholm,1,False
1198,MYH 2024/976,2024,True,Folkuniversitetet - Stiftelsen Kursverksamhete...,GIS Data Mastery: Mobil Insamling och Visualis...,Samhällsbyggnad och byggteknik,30,15,Gävleborg,Gävle,1,False
1199,MYH 2024/977,2024,False,Folkuniversitetet - Stiftelsen Kursverksamhete...,GIS Essentials: Grundläggande tekniker och til...,Samhällsbyggnad och byggteknik,30,0,Gävleborg,Gävle,1,False
1200,MYH 2024/986,2024,True,IHM Business School AB Göteborg,AI-driven marknadsföring och kommunikation,"Ekonomi, administration och försäljning",40,140,Stockholm,Stockholm,1,False
